# Probabilidade de Compra - Target

### Neste exemplo, vamos demonstrar como identificar a probabilidade de um cliente em potencial realizar uma compra, usando de exemplo a empresa Target, que ficou conhecida por prever a gravidez de suas clientes. 

##### Analisaremos os cliques do usuário em links durante a navegação na web para estimar essa chance de compra. Com base nesse nível de interesse, decidiremos se vale a pena entrar em contato com o cliente, por exemplo, enviando um anúncio nas redes sociais oferecendo 10% de desconto no produto.

In [30]:
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
import sklearn.metrics
import sklearn.model_selection
import sklearn.tree
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
db = pd.read_csv(r"C:\Users\Washington\Documents\Iago M\Probabilidade De Compra - Target\target_app_usuarios_v2.csv")

db.dtypes

SESSAO_ID           object
clique_img           int64
ler_avaliacao        int64
ver_categoria        int64
ler_detalhes         int64
ver_video            int64
adicionar_lista      int64
comparar_produto     int64
ver_similar          int64
favoritar            int64
personalizar         int64
comprar              int64
dtype: object

In [12]:
db.head()

,SESSAO_ID,clique_img,ler_avaliacao,ver_categoria,ler_detalhes,ver_video,adicionar_lista,comparar_produto,ver_similar,favoritar,personalizar,comprar
0,sessao_1,0,0,0,0,0,1,0,0,0,0,0
1,sessao_2,1,0,1,0,0,1,1,1,1,0,1
2,sessao_3,1,0,1,0,1,0,0,1,0,0,0
3,sessao_4,0,1,1,0,0,1,0,0,0,0,1
4,sessao_5,0,0,1,0,0,0,0,1,0,0,1


### Estatística Resumida

In [13]:
db.describe()

,clique_img,ler_avaliacao,ver_categoria,ler_detalhes,ver_video,adicionar_lista,comparar_produto,ver_similar,favoritar,personalizar,comprar
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.414000,0.268000,0.524000,0.408000,0.202000,0.274000,0.280000,0.330000,0.174000,0.160000,0.608000
std,0.493042,0.443361,0.499924,0.491955,0.401894,0.446456,0.449449,0.470684,0.379489,0.366973,0.488686
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Analisando Correlação

In [15]:
corr = db.corr()['comprar'] * 100
corr_formatado = corr.apply(lambda x: f"{x:.2f}%")
print(corr_formatado)

clique_img           13.43%
ler_avaliacao         9.74%
ver_categoria        10.42%
ler_detalhes          9.14%
ver_video             9.79%
adicionar_lista      16.26%
comparar_produto     19.96%
ver_similar          11.92%
favoritar            24.97%
personalizar         14.93%
comprar             100.00%
Name: comprar, dtype: object


### Dividindo Métricas para Treinamento e Teste do Modelo

In [26]:
secundarias = db[['clique_img', 'ver_categoria', 'adicionar_lista', 'comparar_produto', 'ver_similar', 'favoritar', 'personalizar']]
principal = db.comprar

In [27]:
sec_treino, sec_teste, prin_treino, prin_teste = train_test_split(secundarias, principal, test_size=0.3)

print("Métrica – Treinamento : ", sec_treino.shape, "Métrica – Teste : ", sec_teste.shape)

Métrica – Treinamento :  (350, 7) Métrica – Teste :  (150, 7)


### Construindo Modelo e Conferindo Precisão

In [28]:
from sklearn.naive_bayes import GaussianNB

classificador = GaussianNB()
classificador = classificador.fit(sec_treino, prin_treino)

previsao = classificador.predict(sec_teste)

# Análise de precisão da previsão
sklearn.metrics.confusion_matrix(prin_teste, previsao) 


array([[43, 15],
       [37, 55]], dtype=int64)

In [33]:
accuracy = accuracy_score(prin_teste, previsao)
print(f"Acurácia do modelo: {accuracy:.2%}")

Acurácia do modelo: 65.33%


### Qual a chance de cada usuário?

In [36]:
pred_prob = (classificador.predict_proba(sec_teste))
pred_prob[0, 1]

print(f"{pred_prob[0, 1]:.2f}")

0.85


### Previsões em tempo real

Agora que o modelo foi construído, vamos usá-lo para previsões em tempo real. Assim, quando o cliente começa a visitar as páginas uma por uma, coletamos essa lista e a usamos para calcular a probabilidade. Fazemos isso para cada novo clique que chega.

In [39]:
dado_navegando = np.array([0,0,0,0,0,0,0]).reshape(1, -1)

probabilidade = classificador.predict_proba(dado_navegando)[:, 1][0]
print(f"Propensão do Novo Usuário: {probabilidade:.2%}")

Propensão do Novo Usuário: 9.03%


C:\Users\Washington\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


Isso quer dizer que o usuário tem uma probabilidade inicial de 9%, agora vamosver a probabilidade se ele apenas adicionar a lista:

In [47]:
dado_navegando = np.array([0,0,1,0,0,0,0]).reshape(1, -1)

probabilidade = classificador.predict_proba(dado_navegando)[:, 1][0]
print(f"Propensão do Novo Usuário: {probabilidade:.2%}")

Propensão do Novo Usuário: 26.67%


C:\Users\Washington\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


### Agora sim, conseguimos calcular a Probabilidade de cada usuário no final da sua sessão.